# Fungut

# 00 Packages and Directory

In [2]:
import IPython

import pandas as pd
import matplotlib.pyplot as plt
import qiime2 as q2
import seaborn as sns
from qiime2 import Visualization


import os

import matplotlib.pyplot as plt
%matplotlib inline

# 01 Data import

In [3]:
PATH = "fungut_data/fungut_metadata.tsv"

In [4]:
surveys_df = pd.read_csv(PATH, sep="\t")

In [5]:
data_dir = 'fungut_data'

In [6]:
!qiime tools peek $data_dir/fungut_forward_reads.qza

UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [21]:
!qiime demux summarize \
  --i-data $data_dir/fungut_forward_reads.qza \
  --o-visualization $data_dir/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: final/fungut_data/demux_summary.qzv


In [9]:
Visualization.load(f"{data_dir}/demux_summary.qzv")

<visualization: Visualization uuid: f3cf44e4-fbc6-4d58-8edc-c3ecd63e6329>

# 02 Trimming the primers

In [7]:
!qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir/fungut_forward_reads.qza \
  --p-front CTTGGTCATTTAGAGGAAGTAA \
  --o-trimmed-sequences $data_dir/fungut_forward_reads_trimmed.qza \
  --verbose

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt -u 0 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 1 -q 0,0 --quality-base 33 --cores 1 -o /tmp/qiime2/jovyan/processes/413-1761295496.77@jovyan/tmp/q2-OutPath-sbihn6ux/ERR5327198_01_L001_R1_001.fastq.gz --front CTTGGTCATTTAGAGGAAGTAA /tmp/qiime2/jovyan/data/3638611d-1767-413b-9390-70ee3d78e4ff/data/ERR5327198_01_L001_R1_001.fastq.gz

This is cutadapt 5.1 with Python 3.10.14
Command line parameters: -u 0 --

# 03 Denoising

1. Denoising Versuch mit trim left 0 und trunc len 0

In [12]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/dada2_rep_seqs_1.qza \
    --o-table $data_dir/dada2_table_1.qza \
    --o-denoising-stats $data_dir/dada2_stats_1.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: fungut_data/dada2_table_1.qza
Saved FeatureData[Sequence] to: fungut_data/dada2_rep_seqs_1.qza
Saved SampleData[DADA2Stats] to: fungut_data/dada2_stats_1.qza


In [5]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats_1.qza \
    --o-visualization $data_dir/dada2_stats_1.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
^C
R[write to console]: 

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/q2cli/util.py", line 275, in get_plugin_manager
    return qiime2.sdk.PluginManager.reuse_existing()
  File "/opt/conda/lib/python3.10/site-packages/qiime2/sdk/plugin_manager.py", line 58, in reuse_existing
    raise UninitializedPluginManagerError
qiime2.sdk.plugin_manager.UninitializedPluginManagerError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/bin/qiime", line 11, in <module>
    sys.exit(qiime())
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1161, in

In [6]:
Visualization.load(f"{data_dir}/dada2_stats_1.qzv")

<visualization: Visualization uuid: 0be95d0b-691d-4067-9bf4-7a742c4460a8>

In [25]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_rep_seqs_1.qza \
    --o-visualization $data_dir/dada2_rep_seqs_1.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_rep_seqs_1.qzv


In [7]:
Visualization.load(f"{data_dir}/dada2_rep_seqs_1.qzv")

<visualization: Visualization uuid: 7aff86e4-c64c-4acb-a684-425833003074>

In [27]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_1.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table_1.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_table_1.qzv


In [8]:
Visualization.load(f"{data_dir}/dada2_table_1.qzv")

<visualization: Visualization uuid: e2be7ee9-5060-4bcb-948e-d72b8af345f3>

2. Denoising Versuch mit trim left 15 und trunc len 130

In [24]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 15 \
   --p-trunc-len 130 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/dada2_rep_seqs_2.qza \
    --o-table $data_dir/dada2_table_2.qza \
    --o-denoising-stats $data_dir/dada2_stats_2.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: fungut_data/dada2_table_2.qza
Saved FeatureData[Sequence] to: fungut_data/dada2_rep_seqs_2.qza
Saved SampleData[DADA2Stats] to: fungut_data/dada2_stats_2.qza


In [29]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats_2.qza \
    --o-visualization $data_dir/dada2_stats_2.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_stats_2.qzv


In [7]:
Visualization.load(f"{data_dir}/dada2_stats_2.qzv")

<visualization: Visualization uuid: 194341aa-a4a6-4841-bd3e-30a012ab58e5>

In [31]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_rep_seqs_2.qza \
    --o-visualization $data_dir/dada2_rep_seqs_2.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_rep_seqs_2.qzv


In [32]:
Visualization.load(f"{data_dir}/dada2_rep_seqs_2.qzv")

<visualization: Visualization uuid: d284a948-3cd7-469f-9327-7abb760e1706>

In [33]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_2.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table_2.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_table_2.qzv


In [12]:
Visualization.load(f"{data_dir}/dada2_table_2.qzv")

<visualization: Visualization uuid: 3e90f270-885e-47ea-a7e8-6d605b846014>

3. Denoising Versuch mit trim left 0 und trunc len 0 & mit den trimmed primers

In [8]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads_trimmed.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/dada2_rep_seqs_trimmed.qza \
    --o-table $data_dir/dada2_table_trimmed.qza \
    --o-denoising-stats $data_dir/dada2_stats_trimmed.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: fungut_data/dada2_table_trimmed.qza
Saved FeatureData[Sequence] to: fungut_data/dada2_rep_seqs_trimmed.qza
Saved SampleData[DADA2Stats] to: fungut_data/dada2_stats_trimmed.qza


In [9]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_rep_seqs_trimmed.qza \
    --o-visualization $data_dir/dada2_rep_seqs_trimmed.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_rep_seqs_trimmed.qzv


In [10]:
Visualization.load(f"{data_dir}/dada2_rep_seqs_trimmed.qzv")

<visualization: Visualization uuid: e0bab21e-e13c-41a4-ae8c-ac5135c8be03>

In [13]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_trimmed.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table_trimmed.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_table_trimmed.qzv


In [14]:
Visualization.load(f"{data_dir}/dada2_table_trimmed.qzv")

<visualization: Visualization uuid: 78c44942-2f7e-478e-bb54-5ad4935eb142>

In [15]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats_trimmed.qza \
    --o-visualization $data_dir/dada2_stats_trimmed.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_stats_trimmed.qzv


In [16]:
Visualization.load(f"{data_dir}/dada2_stats_trimmed.qzv")

<visualization: Visualization uuid: eeaedee0-d2bc-4b10-bf06-6e13126c1ead>

 **Conclusion**
 
Comparing denoised data 1&2:
ITS sequences are more variable in length than 16S data. There is the risk of cutting the truncation length too short and loose valid ITS reads. Regarding that the the original data was good overall (everywhere over 30), it is better to keep all sequences. With this conclusion, it is decided to continue further steps with the data optained by run 1.

--> The trimming step was conducted before denoising. The results with and without trimming are the same. The trimming step is kept anyways. Therefore we continue with the trimmed data and with the denoised data by run 1. 

# 04 Taxonomy

In [ ]:
Taxonomy mit Euler

In [ ]:
Pretrained classifier
- mit eukaryotes
- https://github.com/colinbrislawn/unite-train/releases

In [ ]:
euler:
    miniconda und chiime auf euler laden
    jetzt könne wir mit euler kommunizieren
    für jeden job die parameter definieren + wo sollen datein abgespeichert werden
    laufen lassen
    Hat es funktioniert?

In [ ]:
conda 
-> ampicon distribution (nicht m)
transferring files
zeit: 24h
cpu: nach datenbank orientieren -> etwas mehr als das
